### =======================================================================
### IMPORTACIÓN DE DATOS.
### =======================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import xgboost as xgb

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=np.nan)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES EVENTOS.
eventos = pd.read_csv('events_up_to_01062018.csv', encoding = 'utf-8')

### =======================================================================
### ARMADO DE FEATURES.
### =======================================================================\

In [3]:
## PASAMOS LAS COLUMNAS QUE TIENEN UNA CANTIDAD DE VALORES LIMITADA A UN TIPO CATEGORY
eventos['event'] = eventos['event'].astype('category')
eventos['search_engine'] = eventos['search_engine'].astype('category')
eventos['channel'] = eventos['channel'].astype('category')
eventos['new_vs_returning'] = eventos['new_vs_returning'].astype('category')
eventos['device_type'] = eventos['device_type'].astype('category')
eventos['color'] = eventos['color'].astype('category')
eventos['region'] = eventos['region'].astype('category')
#eventos['city'] = eventos['city'].astype('category')
eventos['browser_version'] = eventos['browser_version'].astype('category')
eventos['screen_resolution'] = eventos['screen_resolution'].astype('category')
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'], infer_datetime_format=True)

In [4]:
## ORDENAMOS LOS DATOS ṔOR PERSONAS EN PRIMER LUGAR Y TIEMPO EN SEGUNDO.
eventos.sort_values(['person', 'timestamp'], ascending=[True, True], inplace=True)

In [5]:
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
eventos['mes'] = eventos['timestamp'].dt.month
eventos['dia'] = eventos['timestamp'].dt.day
eventos['hora'] = eventos['timestamp'].dt.hour

In [6]:
## ARMAMOS UNA COLUMNA PARA EL DÍA DE LA SEMANA COMO NOMBRE.
eventos['diasemana'] = eventos['timestamp'].dt.weekday_name
## PONEMOS LOS NOMBRES DE MANERA MÁS PROLIJA PARA LOS GRÁFICOS.
eventos.loc[eventos.diasemana.str.contains('Monday', na=False), 'diasemana'] = 'lunes'
eventos.loc[eventos.diasemana.str.contains('Tuesday', na=False), 'diasemana'] = 'martes'
eventos.loc[eventos.diasemana.str.contains('Wednesday', na=False), 'diasemana'] = 'miercoles'
eventos.loc[eventos.diasemana.str.contains('Thursday', na=False), 'diasemana'] = 'jueves'
eventos.loc[eventos.diasemana.str.contains('Friday', na=False), 'diasemana'] = 'viernes'
eventos.loc[eventos.diasemana.str.contains('Saturday', na=False), 'diasemana'] = 'sabado'
eventos.loc[eventos.diasemana.str.contains('Sunday', na=False), 'diasemana'] = 'domingo'

In [7]:
# AGREGAMOS UNA COLUMNA PARA INDICAR SI EL EVENTO OCURRIO UN FIN DE SEMANA
eventos['evento_en_finde'] = 0
eventos.loc[(eventos.diasemana.str.contains('DOM', na=False) | eventos.diasemana.str.contains('SAB', na=False)), 'evento_en_finde'] = 1

In [8]:
# DEFINIMOS EL MES COMO NOMBRE PARA FACILITAR LAS COLUMNAS
eventos['mesMayus'] = ''
eventos.loc[eventos.mes == 1, 'mesMayus'] = 'enero'
eventos.loc[eventos.mes == 2, 'mesMayus'] = 'febrero'
eventos.loc[eventos.mes == 3, 'mesMayus'] = 'marzo'
eventos.loc[eventos.mes == 4, 'mesMayus'] = 'abril'
eventos.loc[eventos.mes == 5, 'mesMayus'] = 'mayo'
eventos.loc[eventos.mes == 6, 'mesMayus'] = 'junio'
eventos.loc[eventos.mes == 7, 'mesMayus'] = 'julio'
eventos.loc[eventos.mes == 8, 'mesMayus'] = 'agosto'
eventos.loc[eventos.mes == 9, 'mesMayus'] = 'septiembre'
eventos.loc[eventos.mes == 10, 'mesMayus'] = 'octubre'
eventos.loc[eventos.mes == 11, 'mesMayus'] = 'noviembre'
eventos.loc[eventos.mes == 12, 'mesMayus'] = 'diciembre'

In [9]:
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
eventos['hora_madrugada'] = 0
eventos.loc[((eventos.hora > -1) & (eventos.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
eventos['hora_maniana'] = 0
eventos.loc[((eventos.hora > 6) & (eventos.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
eventos['hora_almuerzo'] = 0
eventos.loc[((eventos.hora > 11) & (eventos.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
eventos['hora_tarde'] = 0
eventos.loc[((eventos.hora > 13) & (eventos.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
eventos['hora_noche'] = 0
eventos.loc[((eventos.hora > 18) & (eventos.hora < 24)), 'hora_noche'] = 1
# TRANSFORMAMOS EN CATEGÓRICAS EL DÍA DE LA SEMANA Y EL MES.
eventos['diasemana'] = eventos['diasemana'].astype('category')
eventos['mesMayus'] = eventos['mesMayus'].astype('category')

In [10]:
## CREAMOS UNA COLUMNA CON CONTENIDO VACIO.
eventos['sistema'] = 'OtrosSistemas'
## SEGÚN QUE FAMILIA DE SO POSEA ACTUALIZAMOS NUESTRA NUEVA COLUMNA.
eventos.loc[eventos.operating_system_version.str.contains('Mac', na=False), 'sistema'] = 'MacOS'
eventos.loc[eventos.operating_system_version.str.contains('iOS', na=False), 'sistema'] = 'iOS'
eventos.loc[eventos.operating_system_version.str.contains('Chrome', na=False), 'sistema'] = 'Chrome'
eventos.loc[eventos.operating_system_version.str.contains('Tizen', na=False), 'sistema'] = 'Tizen'
eventos.loc[eventos.operating_system_version.str.contains('Android', na=False), 'sistema'] = 'Android'
eventos.loc[eventos.operating_system_version.str.contains('Windows Phone', na=False), 'sistema'] = 'Windows Phone'
eventos.loc[eventos.operating_system_version.str.contains('Ubuntu', na=False), 'sistema'] = 'Ubuntu'
eventos.loc[eventos.operating_system_version.str.contains('Linux', na=False), 'sistema'] = 'Linux'
## CASO PARTICULAR, WINDOWS Y WINDOWS PHONE COMPARTEN LA PALABRA.
eventos.loc[(eventos.operating_system_version.str.contains('Windows', na=False) & ~eventos.operating_system_version.str.contains('Phone', na=False)), 'sistema'] = 'Windows'
## AHORA SI PASAMOS LA COLUMNA DE SISTEMAS OPERATIVOS A UN ENUMERATIVO.
eventos['operating_system_version'] = eventos['operating_system_version'].astype('category')
eventos['sistema'] = eventos['sistema'].astype('category')

In [11]:
## AGREGAMOS UNA COLUMNA PARA DIFERENCIAR LAS MARCAS.
eventos['marca'] = 'OtrasMarcas'
## VAMOS OBTENIENDO LAS MARCAS SEGÚN LAS PALABRAS CLAVES QUE LAS CONFORMAN.
eventos.loc[eventos.model.str.contains('Samsung', na=False), 'marca'] = 'Samsung'
eventos.loc[eventos.model.str.contains('Motorola', na=False), 'marca'] = 'Motorola'
eventos.loc[eventos.model.str.contains('Sony', na=False), 'marca'] = 'Sony'
eventos.loc[eventos.model.str.contains('LG ', na=False), 'marca'] = 'LG'
eventos.loc[eventos.model.str.contains('iPad', na=False), 'marca'] = 'iPad'
eventos.loc[eventos.model.str.contains('Asus', na=False), 'marca'] = 'Asus'
eventos.loc[eventos.model.str.contains('iPhone', na=False), 'marca'] = 'iPhone'
eventos.loc[eventos.model.str.contains('Quantum', na=False), 'marca'] = 'Quantum'
eventos.loc[eventos.model.str.contains('Lenovo', na=False), 'marca'] = 'Lenovo'
## AHORA SI PASAMOS LA COLUMNA DE MODELOS A UN ENUMERATIVO.
eventos['model'] = eventos['model'].astype('category')
eventos['marca'] = eventos['marca'].astype('category')

In [12]:
## AGREGAMOS UNA COLUMNA PARA DIFERENCIAR BRASIL DEL RESTO.
# eventos['pais'] = 'OtrosPaises'
# eventos.loc[eventos.country.str.contains('Brazil', na=False), 'pais'] = 'Brasil'
# ## AHORA SI PASAMOS LA COLUMNA DE PAISES A UN ENUMERATIVO.
# eventos['pais'] = eventos['pais'].astype('category')
# eventos['country'] = eventos['country'].astype('category')
eventos['pais'] = 'OtrosPaises'
paisesMasPopulares = eventos['country'].value_counts().to_frame().reset_index().head()['index']
eventos.loc[eventos['country'].isin(paisesMasPopulares), 'pais'] = eventos['country']
eventos['pais'] = eventos['pais'].astype('category')
eventos['pais'].value_counts()

OtrosPaises      2137877
Brazil            197699
Unknown             5273
United States        634
Argentina            123
Canada                75
Name: pais, dtype: int64

In [13]:
## COLUMNA DE CIUDAD
eventos['ciudad'] = 'OtraCiudad'
ciudadesMasPopulares = eventos['city'].value_counts().to_frame().reset_index().head(200)['index']
eventos.loc[eventos['city'].isin(ciudadesMasPopulares), 'ciudad'] = eventos['city']
eventos['ciudad'] = eventos['ciudad'].astype('category')

In [14]:
## COLUMNA DE NAVEGADOR
eventos['navegador'] = 'OtroNavegador'
navegadoresMasPopulares = eventos['browser_version'].value_counts().to_frame().reset_index().head(50)['index']
eventos.loc[eventos['browser_version'].isin(navegadoresMasPopulares), 'navegador'] = eventos['browser_version']
eventos['navegador'] = eventos['navegador'].astype('category')

In [15]:
eventos['Capacidad_en_GB'] = 0
eventos.loc[eventos.storage.str.contains('16GB', na=False), 'Capacidad_en_GB'] = 16
eventos.loc[eventos.storage.str.contains('32GB', na=False), 'Capacidad_en_GB'] = 32
eventos.loc[eventos.storage.str.contains('64GB', na=False), 'Capacidad_en_GB'] = 64
eventos.loc[eventos.storage.str.contains('128GB', na=False), 'Capacidad_en_GB'] = 128
eventos.loc[eventos.storage.str.contains('8GB', na=False), 'Capacidad_en_GB'] = 8
eventos.loc[eventos.storage.str.contains('256GB', na=False), 'Capacidad_en_GB'] = 256
eventos.loc[eventos.storage.str.contains('4GB', na=False), 'Capacidad_en_GB'] = 4
eventos.loc[eventos.storage.str.contains('512MB', na=False), 'Capacidad_en_GB'] = 0.512
eventos['storage'] = eventos['storage'].astype('category')

In [16]:
## CREAMOS UNA COLUMNA PARA DIFERENCIAR LA CONDICION.
eventos['Puntaje'] = 0
## ABSTRAEMOS SOLO AQUELLOS QUE TIENEN UNA CONDICIÓN ASOCIADA Y LO PASAMOS A NUESTRO IDIOMA.
eventos.loc[eventos.condition.str.contains('Excelente', na=False), 'Puntaje'] = 4
eventos.loc[eventos.condition.str.contains('Muito Bom', na=False), 'Puntaje'] = 3
eventos.loc[eventos.condition.str.contains('Novo', na=False), 'Puntaje'] = 5
## SOLO NOS QUEDAN DOS TIPOS DE EVENTOS QUE SON REFERENCIA A LOS 'BUENOS'
eventos.loc[eventos.Puntaje == 0, 'Puntaje'] = 2
eventos['condition'] = eventos['condition'].astype('category')

In [17]:
# COMO TENEMOS UN EVENTO CON EL MISMO NOMBRE SE GENERA CONFLICTOS, 
# ERGO LE MODIFICAMOS EL NOMBRE PARA NO TENER DOS COLUMNAS CON = NOMBRE Y DISTINTO TIPO.
eventos.rename(columns={'staticpage': 'Genstatpage'}, inplace=True)
aux = eventos


In [18]:
eventos = aux

In [19]:
# COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)
##################### DIA DE SEMANA
#dummies = pd.get_dummies(eventos['diasemana'], drop_first=False)
#eventos = pd.concat([eventos, dummies], axis=1)
##################### MES
dummies = pd.get_dummies(eventos['mesMayus'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### TIPOS DE EVENTOS
dummies = pd.get_dummies(eventos['event'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### ALMACENAMIENTO
dummies = pd.get_dummies(eventos['storage'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### CONDICION DEL DISPOSITIVO
dummies = pd.get_dummies(eventos['condition'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### USUARIO NUEVO O NO
dummies = pd.get_dummies(eventos['new_vs_returning'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### SISTEMA OPERATIVO
dummies = pd.get_dummies(eventos['sistema'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### MARCA
dummies = pd.get_dummies(eventos['marca'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
##################### PAIS DE EVENTO
dummies = pd.get_dummies(eventos['pais'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
eventos.rename(columns={'Unknown': 'Unknown_country'}, inplace=True)
##################### TIPO DE DISPOSITIVO
dummies = pd.get_dummies(eventos['device_type'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
eventos.rename(columns={'Unknown': 'Unknown_device_type'}, inplace=True)
##################### CIUDAD
#dummies = pd.get_dummies(eventos['ciudad'], drop_first=False)
#eventos = pd.concat([eventos, dummies], axis=1)
#eventos.drop(['Unknown'], axis=1)
##################### NAVEGADOR
dummies = pd.get_dummies(eventos['navegador'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
#####################


In [20]:
eventos.rename(columns={'generic listing': 'geneList', 'staticpage': 'statpage', 'Unknown': 'Unknown_city', 'staticpage': 'SP'}, inplace=True)

In [21]:
columnas_relevantes = list(eventos.select_dtypes(include=['int','float64','uint8']).columns)

columnas_relevantes.remove('sku')
columnas_relevantes.append('person')
eventos_filtrados = eventos.loc[:, eventos.columns.isin(columnas_relevantes)]
columnas_relevantes.remove('person')

eventos_por_usuario = eventos_filtrados.groupby('person')[columnas_relevantes].sum().astype(np.float16).reset_index()


In [ ]:
eventos_por_usuario.head()

In [ ]:
columnas_relevantes

### =======================================================================
### ENTRENAMIENTO Y PREDICCIÓN.
### =======================================================================

In [ ]:
## OBTENEMOS TODA LA INFORMACIÓN DEL SET DE ENTRENAMIENTO.
y_train = pd.read_csv('labels_training_set.csv', encoding = 'utf-8')
test_users = pd.read_csv('trocafone_kaggle_test.csv', encoding = 'utf-8')

In [ ]:
## Filtramos los eventos para los usuarios que se encuentran en el set de entrenamiento
train = pd.merge(eventos_por_usuario, y_train, on='person', how='inner')
test = pd.merge(eventos_por_usuario, test_users)

In [ ]:
features = list(columnas_relevantes)
x = train[features]
y = train['label']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state = 1)

### =======================================================================
### PARAMETER TUNING
### =======================================================================

In [ ]:
#data = pd.DataFrame(columns=['n_estimator', 'learning_rate', 'prediccion'])
#for estimators in range(100, 300, 5):
#     for i in range(1, 11, 1):
#         learning_rate = i / 100.0
#         xgboost = xgb.XGBClassifier(n_estimators=estimators, learning_rate=learning_rate)
#         xgboost.fit(xtrain, ytrain, eval_metric='auc')
#         y_pred_rf = xgboost.predict_proba(xtest)[:,1]
#         df = pd.DataFrame({'n_estimator': [estimators], 'learning_rate': [learning_rate],'prediccion': [metrics.roc_auc_score(ytest, y_pred_rf)]})
#         data = pd.concat([data,df])
        

In [ ]:
# data0 = pd.DataFrame(columns=['depth', 'prediccion'])
# for depth in range(2, 15):
#     xgboost = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=depth)
#     xgboost.fit(xtrain, ytrain, eval_metric='auc')
#     y_pred_rf = xgboost.predict_proba(xtest)[:,1]
#     df = pd.DataFrame({'depth': [depth],'prediccion': [metrics.roc_auc_score(ytest, y_pred_rf)]})
#     data0 = pd.concat([data,df])
    

In [ ]:
# sns.set(style="darkgrid")

# # Tamaño para hojas A4.
# fig, ax = plt.subplots()
# fig.set_size_inches(11.7, 8.27)

# data1 = data[data['learning_rate'] == 0.01]
# data2 = data[data['learning_rate'] == 0.03]
# data3 = data[data['learning_rate'] == 0.05]
# data4 = data[data['learning_rate'] == 0.07]
# data5 = data[data['learning_rate'] == 0.09]
# data6 = data[data['learning_rate'] == 0.1]

# g = sns.lineplot(x="n_estimator", y="prediccion", data=data1, label='learning rate 0.01')
# g = sns.lineplot(x="n_estimator", y="prediccion", data=data2, label='learning rate 0.03')
# g = sns.lineplot(x="n_estimator", y="prediccion", data=data3, label='learning rate 0.05')
# g = sns.lineplot(x="n_estimator", y="prediccion", data=data4, label='learning rate 0.07')
# g = sns.lineplot(x="n_estimator", y="prediccion", data=data5, label='learning rate 0.08')
# g = sns.lineplot(x="n_estimator", y="prediccion", data=data6, label='learning rate 0.1')
# g.set_title("learning rate y n_estimators tuning", fontsize=35)
# g.set_xlabel("n_estimators", fontsize=30)
# g.set_ylabel("Score de prediccion", fontsize=30)

In [ ]:
# sns.set(style="darkgrid")

# # Tamaño para hojas A4.
# fig, ax = plt.subplots()
# fig.set_size_inches(11.7, 8.27)

# g = sns.scatterplot(x="depth", y="prediccion", data=data0)

# g.set_title("depth tuning", fontsize=35)
# g.set_xlabel("depth", fontsize=30)
# g.set_ylabel("Score de prediccion", fontsize=30)

### =======================================================================
### Resultado
### =======================================================================

In [ ]:
xgboost = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=3)
xgboost.fit(x, y)
y_pred_rf = xgboost.predict_proba(test[features])[:,1]

In [ ]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO.
## =================================================================================================
submission = pd.DataFrame({ 'label': y_pred_rf, 'person': test['person'] })
submission.to_csv("submission_grupo17_fin.csv", index=False)

In [ ]:
## SE AGREGÓ EL FEATURE QUE ENGLOBA LOS SISTEMAS OPERATIVOS MÁS COMUNES UTILIZADOS O BUSCADOS POR LOS USUARIOS:
## PREVIAMENTE TENÍAMOS: 0.84077
## RESULTADO DEL SUBMIT: 0.85216 >> VEMOS UNA MEJORA.

In [ ]:
## SE AGREGÓ EL FEATURE QUE ENGLOBA LOS MODELOS MÁS COMUNES UTILIZADOS O BUSCADOS POR LOS USUARIOS:
## PREVIAMENTE TENÍAMOS: 0.85216
## RESULTADO DEL SUBMIT:0.85294 >> VEMOS UNA MEJORA.

In [ ]:
## SE AGREGÓ UN PAR DE FEATURES (pais, device_type, Capacidad_en_GB, Puntaje):
## PREVIAMENTE TENÍAMOS: 0.85294
## RESULTADO DEL SUBMIT: 0.85524 >> VEMOS UNA MEJORA.

In [ ]:
## AHORA LO QUE SE BUSCA ES HACER EL PROMEDIO EN LUGAR DE LA SUMA:
## PREVIAMENTE TENÍAMOS: 0.85524
## RESULTADO DEL SUBMIT: 0.85519 >> CON LO CUAL EMPEORA.

## Tocando los hiperparametros >> 0.85546

In [ ]:
## AGREGAMOS LAS CIUDADES CON MAS EVENTOS
## PREVIAMENTE TENÍAMOS: 0.85546
## RESULTADO DEL SUBMIT: 0.85438 >> CON LO CUAL EMPEORA.